In [35]:
import requests
import pprint
import arrow
import pandas as pd
import json
pp = pprint.PrettyPrinter(indent=2)

In [3]:
base_url = "https://firemap.sdsc.edu:5443/stations?"

In [22]:
sdsc_lat = 32.88437231
sdsc_lon = -117.2414495
## Latitude and Longitude of the center of the bounding box  
params = {
    'minLat': sdsc_lat - 0.075,
    'minLon': sdsc_lon - 0.075,
    'maxLat': sdsc_lat + 0.075,
    'maxLon': sdsc_lon + 0.075,
    'selection': 'boundingBox'
}
now = arrow.Arrow.now()


## Fill in a string to append to the URL and from a box around the chosen Latitude and Longitude

boundingBoxURL= base_url
## concatenate strings to form a final URL
data = requests.get(base_url, params=params).json()

observable_set = set()
features = data['features']
for feature in features:
    properties = feature['properties']
    for observable in properties.keys():
        if observable not in ['description', \
                              'distanceFromLocation', \
                             'latest-images']:
            observable_set.add(observable)

In [32]:
metadata_dict = dict()
for observable in observable_set:
    baseURL = "https://firemap.sdsc.edu:5443/stations/data?"
    to = arrow.Arrow.now()
    frm = to.replace(days=-14)
    #selectionType="selection=closestTo"
    selectionType = "selection=boundingBox"
    #selectionParameters = "&lat=%s&lon=%s" % (str(lat),str(lon))
    selectionParameters = "&minLat=%s&minLon=%s&maxLat=%s&maxLon=%s" \
                            % ( str(params['minLat']), str(params['minLon']),\
                               str(params['maxLat']), str(params['maxLon']) )
    urlDateTime = "&from=%s&to=%s" % ( str(frm) , str(to) )
    observableInfo = "&observable=" + observable
    urlPlot = baseURL \
            + selectionType \
            + selectionParameters \
            + observableInfo \
            + urlDateTime
    data = requests.get(urlPlot).json()
    features = data['features']
    for feature in features:
        metadata = {
            'latitude': feature['geometry']['coordinates'][1],
            'longitude': feature['geometry']['coordinates'][0],
            'point_type': observable,
            'unit': feature['properties']['units'][observable]
        }
        value_list = feature['properties'][observable]
        time_list = feature['properties']['timestamp']
        point_name = feature['properties']['description']['name'] + '_' + observable
        pd.Series(index=time_list, data=value_list) \
            .to_csv('data/%s.csv' % point_name, index_label='time', header=['value'])
        metadata_dict[point_name] = metadata

In [36]:
with open('metadata/wifier_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp)